In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
import os
from PIL import Image
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
from tqdm.notebook import tqdm
print("PyTorch Version: ",torch.__version__)
print("Torchvision Version: ",torchvision.__version__)

SEED = 1234
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)

PyTorch Version:  1.3.1
Torchvision Version:  0.4.2


In [2]:
def parse_annotation(path_annotation):
        f = open(path_annotation, 'r')
        content = f.read()
        infos = content.split(',')

        label = infos[0]
        label = label.replace('"', "").replace('[', "")

        xmin = int(infos[1].replace("[", "").lstrip())
        ymin = int(infos[2].lstrip())
        xmax = int(infos[3].lstrip())
        ymax = int(infos[4].replace("]", "").lstrip())

        f.close()
        return label, [xmin, ymin, xmax, ymax]

class LearningRCNNDataset(Dataset):
    def __init__(self, root, transforms=None):
        self.root = root
        self.transforms = transforms
        # load all image files, sorting them to
        # ensure that they are aligned

        self.files_imgs = list(sorted(filter(lambda x: x.split('_')[1] in keep_ids, [img_file for img_file in os.listdir(root) if img_file.endswith(".jpg")])))
        self.files_annotation = list(sorted(filter(lambda x: x.split('_')[1] in keep_ids, [img_file for img_file in os.listdir(root) if img_file.endswith(".txt")])))
        self.label_parsor = {"TemoinNone": 0,
                            "Temoin25": 1,
                            "Temoin50": 2,
                            "Temoin75": 3,
                            "Temoin100": 4}
        
    def __getitem__(self, idx):
        # load images ad masks
        path_img = os.path.join(self.root, self.files_imgs[idx])
        path_annotation = os.path.join(self.root, self.files_annotation[idx])
        img = Image.open(path_img)  # .convert("RGB")
        label, bboxe = parse_annotation(path_annotation=path_annotation)

        if self.transforms:
            img, bboxe = self.transforms(img, bboxe)
        if bboxe[0] == bboxe[2] or bboxe[1] == bboxe[3]:
            area = 0
        else:
            area = (bboxe[2] - bboxe[0] + 1) * bboxe[3] - bboxe[1] + 1
        return img , {"boxes": torch.tensor(bboxe, dtype=torch.float),
                      "labels": torch.tensor([self.label_parsor[label]], dtype=torch.int64),
                      "image_id": torch.tensor([idx], dtype=torch.int64),
                      "area": torch.tensor([area]),
                      "iscrowd": torch.tensor([True])}

    def __len__(self):
        return len(self.files_imgs)


class LearningCNNMichelinDataset(Dataset):
    # keep_ids gives the indices of train or test images
    def __init__(self, root, keep_ids, transforms=None):
        self.root = root
        self.transforms = transforms
        self.files_imgs = list(sorted([img_file for img_file in os.listdir(root) if img_file.endswith(".jpg")]))
        self.files_annotation = list(sorted([img_file for img_file in os.listdir(root) if img_file.endswith(".txt")]))
        self.label_parsor = {"TemoinNone": 0,
                            "Temoin25": 1,
                            "Temoin50": 2,
                            "Temoin75": 3,
                            "Temoin100": 4}
        
    def __getitem__(self, idx):
        # load images ad masks
        path_img = os.path.join(self.root, self.files_imgs[idx])
        path_annotation = os.path.join(self.root, self.files_annotation[idx])
        img = Image.open(path_img)  # .convert("RGB")
        label, _ = parse_annotation(path_annotation=path_annotation)
        if self.transforms:
            img = self.transforms(img)
        return img, torch.tensor(self.label_parsor[label])

    def __len__(self):
        return len(self.files_imgs)

In [3]:
def train_model(model, dataloaders, criterion, optimizer, num_epochs=25, is_inception=False):
    since = time.time()

    val_acc_history = []

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in tqdm(dataloaders[phase]):
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    # Get model outputs and calculate loss
                    # Special case for inception because in training it has an auxiliary output. In train
                    #   mode we calculate the loss by summing the final output and the auxiliary output
                    #   but in testing we only consider the final output.
                    if is_inception and phase == 'train':
                        # From https://discuss.pytorch.org/t/how-to-optimize-inception-model-with-auxiliary-classifiers/7958
                        outputs, aux_outputs = model(inputs)
                        loss1 = criterion(outputs, labels)
                        loss2 = criterion(aux_outputs, labels)
                        loss = loss1 + 0.4*loss2
                    else:
                        outputs = model(inputs)
                        loss = criterion(outputs, labels)

                    _, preds = torch.max(outputs, 1)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
            if phase == 'val':
                val_acc_history.append(epoch_acc)

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, val_acc_history

In [4]:
def initialize_model(model_name, num_classes, feature_extract, use_pretrained=True):
    # Initialize these variables which will be set in this if statement. Each of these
    #   variables is model specific.
    model_ft = None
    input_size = 0

    if model_name == "resnet":
        """ Resnet18
        """
        model_ft = models.resnet18(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.fc.in_features
        model_ft.fc = nn.Linear(num_ftrs, num_classes)
        input_size = 224

    elif model_name == "alexnet":
        """ Alexnet
        """
        model_ft = models.alexnet(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.classifier[6].in_features
        model_ft.classifier[6] = nn.Linear(num_ftrs,num_classes)
        input_size = 224

    elif model_name == "vgg":
        """ VGG11_bn
        """
        model_ft = models.vgg11_bn(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.classifier[6].in_features
        model_ft.classifier[6] = nn.Linear(num_ftrs,num_classes)
        input_size = 224

    elif model_name == "squeezenet":
        """ Squeezenet
        """
        model_ft = models.squeezenet1_0(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        model_ft.classifier[1] = nn.Conv2d(512, num_classes, kernel_size=(1,1), stride=(1,1))
        model_ft.num_classes = num_classes
        input_size = 224

    elif model_name == "densenet":
        """ Densenet
        """
        model_ft = models.densenet121(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.classifier.in_features
        model_ft.classifier = nn.Linear(num_ftrs, num_classes)
        input_size = 224

    elif model_name == "inception":
        """ Inception v3
        Be careful, expects (299,299) sized images and has auxiliary output
        """
        model_ft = models.inception_v3(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        # Handle the auxilary net
        num_ftrs = model_ft.AuxLogits.fc.in_features
        model_ft.AuxLogits.fc = nn.Linear(num_ftrs, num_classes)
        # Handle the primary net
        num_ftrs = model_ft.fc.in_features
        model_ft.fc = nn.Linear(num_ftrs,num_classes)
        input_size = 299

    else:
        print("Invalid model name, exiting...")
        exit()

    return model_ft, input_size

def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False

In [5]:
#data_dir = "./data/training_dataset"
data_dir = "./data/data_classic"

# Models to choose from [resnet, alexnet, vgg, squeezenet, densenet, inception]
model_name = "squeezenet"

# Number of classes in the dataset
num_classes = 5

# Batch size for training (change depending on how much memory you have)
batch_size = 64

# Number of epochs to train for
num_epochs = 15

# Flag for feature extracting. When False, we finetune the whole model,
#   when True we only update the reshaped layer params
feature_extract = False

# Detect if we have a GPU available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

num_workers = 4

In [6]:
# Initialize the model for this run
model_ft, input_size = initialize_model(model_name, num_classes, feature_extract, use_pretrained=True)

# Print the model we just instantiated
print(model_ft)

SqueezeNet(
  (features): Sequential(
    (0): Conv2d(3, 96, kernel_size=(7, 7), stride=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
    (3): Fire(
      (squeeze): Conv2d(96, 16, kernel_size=(1, 1), stride=(1, 1))
      (squeeze_activation): ReLU(inplace=True)
      (expand1x1): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1))
      (expand1x1_activation): ReLU(inplace=True)
      (expand3x3): Conv2d(16, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (expand3x3_activation): ReLU(inplace=True)
    )
    (4): Fire(
      (squeeze): Conv2d(128, 16, kernel_size=(1, 1), stride=(1, 1))
      (squeeze_activation): ReLU(inplace=True)
      (expand1x1): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1))
      (expand1x1_activation): ReLU(inplace=True)
      (expand3x3): Conv2d(16, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (expand3x3_activation): ReLU(inplace=True)
    )
    (5): Fire(
   

In [7]:
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(input_size),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(input_size),
        transforms.CenterCrop(input_size),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

In [8]:
# Take the ids for the train and the valid set
L_ids = list(set([img_file.split("_")[1] for img_file in os.listdir(data_dir) if img_file.endswith(".jpg")]))
np.random.shuffle(L_ids)
length = len(L_ids)
train_ratio = 0.75
training_length = int(length * train_ratio)
L_ids_train = L_ids[:training_length]
L_ids_valid = L_ids[training_length:]

In [9]:
dataset_train = LearningCNNMichelinDataset(data_dir, L_ids_train, transforms=data_transforms["train"])
dataset_valid = LearningCNNMichelinDataset(data_dir, L_ids_valid, transforms=data_transforms["val"])

In [10]:
dataloaders_dict = {"train": DataLoader(dataset_train, batch_size=batch_size, shuffle=True, num_workers=num_workers),
                    "val": DataLoader(dataset_valid, batch_size=batch_size, shuffle=True, num_workers=num_workers)}

In [11]:
model_ft = model_ft.to(device)

In [12]:
# Gather the parameters to be optimized/updated in this run. If we are
#  finetuning we will be updating all parameters. However, if we are
#  doing feature extract method, we will only update the parameters
#  that we have just initialized, i.e. the parameters with requires_grad
#  is True.
params_to_update = model_ft.parameters()
print("Params to learn:")
if feature_extract:
    params_to_update = []
    for name,param in model_ft.named_parameters():
        if param.requires_grad == True:
            params_to_update.append(param)
            print("\t",name)
else:
    for name,param in model_ft.named_parameters():
        if param.requires_grad == True:
            print("\t",name)

Params to learn:
	 features.0.weight
	 features.0.bias
	 features.3.squeeze.weight
	 features.3.squeeze.bias
	 features.3.expand1x1.weight
	 features.3.expand1x1.bias
	 features.3.expand3x3.weight
	 features.3.expand3x3.bias
	 features.4.squeeze.weight
	 features.4.squeeze.bias
	 features.4.expand1x1.weight
	 features.4.expand1x1.bias
	 features.4.expand3x3.weight
	 features.4.expand3x3.bias
	 features.5.squeeze.weight
	 features.5.squeeze.bias
	 features.5.expand1x1.weight
	 features.5.expand1x1.bias
	 features.5.expand3x3.weight
	 features.5.expand3x3.bias
	 features.7.squeeze.weight
	 features.7.squeeze.bias
	 features.7.expand1x1.weight
	 features.7.expand1x1.bias
	 features.7.expand3x3.weight
	 features.7.expand3x3.bias
	 features.8.squeeze.weight
	 features.8.squeeze.bias
	 features.8.expand1x1.weight
	 features.8.expand1x1.bias
	 features.8.expand3x3.weight
	 features.8.expand3x3.bias
	 features.9.squeeze.weight
	 features.9.squeeze.bias
	 features.9.expand1x1.weight
	 features.

In [13]:
optimizer_ft = optim.SGD(params_to_update, lr=0.001, momentum=0.9)

criterion = nn.CrossEntropyLoss()

In [14]:
model_ft, hist = train_model(model_ft, dataloaders_dict, criterion, optimizer_ft, num_epochs=num_epochs, is_inception=(model_name=="inception"))

Epoch 0/14
----------



train Loss: 1.0139 Acc: 0.5814



val Loss: 0.4779 Acc: 0.8272

Epoch 1/14
----------



train Loss: 0.6145 Acc: 0.7710



val Loss: 0.2973 Acc: 0.9017

Epoch 2/14
----------


...


train Loss: 0.4043 Acc: 0.8540



val Loss: 0.1477 Acc: 0.9518

Epoch 4/14
----------



train Loss: 0.3651 Acc: 0.8657



val Loss: 0.2272 Acc: 0.9223

Epoch 5/14
----------


KeyboardInterrupt: 